<a href="https://colab.research.google.com/github/AnnaM-C/computational-intelligence/blob/main/Week9GA_Merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Merging GA with Neural network from week 9

import torch
import torch.nn.functional as F
from numpy import genfromtxt
import matplotlib.pyplot as plt
import numpy as np

import random
from sympy.combinatorics.graycode import GrayCode
from sympy.combinatorics.graycode import gray_to_bin

!pip install deap
from deap import creator, base, tools, algorithms

In [ ]:
# load the training data

# Import the cancer_TR.dat file
from google.colab import files
files.upload()

data = genfromtxt('cancer_TR.dat', delimiter=' ')
x = data[:, 0:9]
y = data[:, 9:11]
x = torch.as_tensor(x, dtype=torch.double)
y = torch.as_tensor(y, dtype=torch.double)

Saving cancer_TR.dat to cancer_TR (5).dat
Saving cancer_tt.dat to cancer_tt (5).dat
torch.Size([525, 9])
tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [1., 0.],
        [1., 0.]], dtype=torch.float64)


In [ ]:
print("Input shape: ", x.shape)
print(x[0])
print(y.shape)

Input shape:  torch.Size([525, 9])
tensor([0.2000, 0.1000, 0.1000, 0.1000, 0.2000, 0.1000, 0.2000, 0.1000, 0.1000],
       dtype=torch.float64)
torch.Size([525, 2])


In [ ]:
# set up the network
class Net(torch.nn.Module):
    # initialise one hidden layer and one output layer
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)  # hidden layer
        self.out = torch.nn.Linear(n_hidden, n_output)  # output layer

    # connect up the layers: the input passes through the hidden, then the sigmoid, then the output layer
    def forward(self, x):
        x = F.relu(self.hidden(x))  # activation function for hidden layer
        print("X: ", x)
        # x = x.unsqueeze(0)
        # x = torch.flatten(x, start_dim=1)
        x=x.view(x.size(0), -1)
        x = self.out(x)
        return x

In [ ]:
# Defining a network to use for the GA algorithm


# There are 2 classes to output and 9 input features
net = Net(n_feature=9, n_hidden=5, n_output=2)  # define the network
print(net)  # net architecture

print((net.hidden.weight).shape)
print((net.out.weight).shape)
print((net.hidden.bias).shape)
print((net.out.bias).shape)

paramCount = sum(param.numel() for param in net.parameters())

print(paramCount) # 62 decision variables

Net(
  (hidden): Linear(in_features=9, out_features=5, bias=True)
  (out): Linear(in_features=5, out_features=2, bias=True)
)
torch.Size([5, 9])
torch.Size([2, 5])
torch.Size([5])
torch.Size([2])
62


In [ ]:
# GA Evaluation function

loss_func = torch.nn.CrossEntropyLoss()

# TODO: FIX FROM:TO for weights

def calcFitness(individual):
  # print("Individual: ", individual)

  weights=separatevariables(individual)
  weights = np.asarray(weights)
  print("Weights: ", len(weights))
  # x = torch.tensor(x, dtype=torch.double)
  # y = torch.tensor(y, dtype=torch.double)
  # print(weights) # Length: 2, but we need more to be able to reshape! Change number of decision vars so it is enough for weights + biases!?

  # First 45 numbers
  net.hidden.weight=torch.nn.Parameter(torch.from_numpy(weights[0:45].reshape(5,9))) # Shape is (5,9)

  net.out.weight=torch.nn.Parameter(torch.from_numpy(weights[45:55].reshape(2,5).T)) # Shape is (2,5)

  net.hidden.bias=torch.nn.Parameter(torch.from_numpy(weights[55:60])) # Shape is 5

  net.out.bias=torch.nn.Parameter(torch.from_numpy(weights[60:])) # Shape is 2

  out = net(x) # input x and predict based on x
  loss = loss_func(out, y)
  return loss.item(),

In [ ]:
# GA Body

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

popSize     = 50 #Population size

#dimension   = 2 #Numer of decision variable x
dimension = 62

numOfBits   = 10 #Number of bits in the decision variable
iterations  = 100 #Number of generations to be run
dspInterval = 10
nElitists   = 1 #number of elite individuals selected
omega       = 5
crossPoints = 2 #variable not used. instead tools.cxTwoPoint
crossProb   = 0.6
flipProb    = 1. / (dimension * numOfBits) #bit mutate prob
mutateprob  = .1 #mutation prob
maxnum      = 2**numOfBits #absolute max size of number coded by binary list 1,0,0,1,1,....

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)

toolbox.register("individual", tools.initRepeat, creator.Individual,
    toolbox.attr_bool, numOfBits*dimension)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", calcFitness) # Change this

toolbox.register("mate", tools.cxTwoPoint)

toolbox.register("mutate", tools.mutFlipBit, indpb=flipProb)

toolbox.register("select", tools.selRoulette, fit_attr='fitness')

# Convert chromosome to real number
# input: list binary 1,0 of length numOfBits representing number using gray coding
# output: real value
def chrom2real(c):
    indasstring=''.join(map(str, c))
    degray=gray_to_bin(indasstring)
    numasint=int(degray, 2) # convert to int from base 2 list
    numinrange=-5+10*numasint/maxnum
    return numinrange


# input: concatenated list of binary variables
# output: tuple of real numbers representing those variables
def separatevariables(v):
    print(len(v))
    result = []

    # Loop through the range of 0 to the length of v with a step of numOfBits
    for i in range(0, len(v), numOfBits):
        # Take a chunk of numOfBits elements from v and apply chrom2real
        chunk_result = chrom2real(v[i:i + numOfBits])
        # Append the result to the final list
        result.append(chunk_result)

    return result

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
def main():
    #random.seed(64)

    # create an initial population of individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=popSize)

#     for individ in pop:
#         sep=separatevariables(individ)
#         print(sep[0],sep[1])

    # print("Pop: ", pop)

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))

    #print(fitnesses)
    for ind, fit in zip(pop, fitnesses):
        #print(ind, fit)
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0

    # Begin the evolution
    while g < iterations:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)
#         for individ in pop:
#             print(individ)

        # Select the next generation individuals
        offspring = tools.selBest(pop, nElitists) + toolbox.select(pop,len(pop)-nElitists)
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

#         for individ in offspring:
#             print(individ)


        # Apply crossover and mutation on the offspring
        # make pairs of offspring for crossing over
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < crossProb:
                #print('before crossover ',child1, child2)
                toolbox.mate(child1, child2)
                #print('after crossover ',child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability mutateprob
            if random.random() < mutateprob:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        #print("  Evaluated %i individuals" % len(invalid_ind))

        # The population is entirely replaced by the offspring
        pop[:] = offspring

        if g%dspInterval ==0:
            # Gather all the fitnesses in one list and print the stats
            fits = [ind.fitness.values[0] for ind in pop]

            length = len(pop)
            mean = sum(fits) / length
            sum2 = sum(x*x for x in fits)
            std = abs(sum2 / length - mean**2)**0.5

            print("  Min %s" % min(fits))
            print("  Max %s" % max(fits))
            print("  Avg %s" % mean)
            print("  Std %s" % std)

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    print("Decoded x1, x2 is %s, %s" % (separatevariables(best_ind)))


if __name__ == '__main__':
  main()

620
Weights:  62
tensor([[0.2000, 0.1000, 0.1000,  ..., 0.2000, 0.1000, 0.1000],
        [0.2000, 0.1000, 0.1000,  ..., 0.3000, 0.1000, 0.1000],
        [0.5000, 0.1000, 0.1000,  ..., 0.2000, 0.1000, 0.1000],
        ...,
        [0.5000, 0.8000, 0.9000,  ..., 0.7000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000],
        [0.4000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000]],
       dtype=torch.float64)
X:  tensor([[ 0.0000,  0.0000,  2.8564,  0.1025,  4.3740],
        [ 0.0000,  0.0000,  3.2363,  0.5498,  4.7939],
        [ 0.0000,  0.0000,  2.3027,  0.0000,  5.3232],
        ...,
        [ 0.0000,  4.9336, 12.4443,  2.9336,  7.1631],
        [ 0.0000,  0.0000,  4.2959,  0.0332,  4.0010],
        [ 0.0000,  0.0000,  2.1074,  0.0000,  4.5869]], dtype=torch.float64,
       grad_fn=<ReluBackward0>)


<ipython-input-77-dad968abd1b6>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xx = torch.tensor(x, dtype=torch.double)
<ipython-input-77-dad968abd1b6>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yy = torch.tensor(y, dtype=torch.double)


RuntimeError: ignored